# Models for the 2AFC auditory triangles task

## Alexandre Filipowicz & Adrian Radillo

### April 19th, 2019

The goal for this notebook is to create working models for the auditory 2AFC task. The goal of the project is to have people behave with different levels of mental model complexity. We will put together a more formal definition in another document, but briefly complexity is being characterized as the amount of past information being encoded to predict the future (cf., Bialek et al., 2001, Neural Computation).

Our main goal is to come up with task conditions that manipulate a person's model complexity while being exposed to very similar stimuli (i.e., use the same stimuli in more/less complex ways).

There are three conditions we would like to consider (at least, as a start):

- **Passive condition:** This is the condition that requires the simplest model as no information needs to be encoded. The person would not have to make any responses, except for maybe catch trials to make sure they are paying attention.

- **Stimulus Identification:** This condition requires a model, but not one that takes too much information. All it does is it identifies, on each trial, the identity of the previous stimulus.

- **Source prediction**: The most complex model would be one in which the person tries to predict the stimulus/source on the next trial. This could be in a condition in which there is or there isn;t a hazard rate (so far these measures have been used in situations where a hazard rate needs to be inferred).

The blocks below are meant to test different models that do different levels of inference:

- **Source inference model:** Model that strictly attempts to infer the source generating the stimuli
- **Source inference with fixed hazard:** Model that infers the source while considering a single, fixed hazard rate
- **Source and hazard inference - fixed hazard:** Model that infers the the source and the hazard rate, assuming that sources can change but not the hazard rate. DOes this inference with a prior over the hazard space.
- **Source and hazard inference - variable hazard:**: Model that infers the source and the hazard rate, but assumes that the hazard rate itself can also change. Also has a prior over the hazard space.

In [48]:
#Import libraries
%pylab
%matplotlib inline
import numpy as np
import pandas as pd
from jupyterthemes import jtplot
jtplot.style()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


### 0) Task set up

To be clear about the task we are attempting to model, the general set up will be that a person hears tone, either in the left ear or the right ear, and has to make inferences about those tones based on the condition they are in.

This may change but depending on the instructions, participants will be told that two sources could be generating the tones:

- Source 1 generates tone 80% of the time on the left and 20 % on the right
- Source 2 generates tones 80% on the right and 20% on the left

Tone produced from each source are generated via stochastic bernoulli process.


### 1) Source inference model

First start with a simple source inference model that does not include a hazard rate. This is a simple bayesian updater that acummlates evidence for one source or the other and makes it's best guess as to which source has generated all previously heard tones on each trial.

**ADD MATH HERE**

Let $z$ be the source and $x$ be the observations on a trial $t$ - our model is simply trying to compute $p(z|x_{1:t})$ using bayes rule:

$$p(z|x_{1:t}) \propto p(x_{1:t}|z)p(z)$$

In [35]:
def source_inf(x,p):
    """
    Source inference model - infers which of two sources generated binary observations x
    This model is predictive
    Arguments:
        - x: array of binary observations (0s and 1s)
        - p: probability p of the bernoulli processes
    Returns:
        - z_hat: trial by trial estimate of source
        - pz: trial by trial probability that each source is the generative source
    """
    # Set up probability distribution over source (z) space
    zspc = np.array([0,1])      #source space (range of possible values)
    pz = np.empty((2,len(x)))   #z by trials matrix to capture posterior as it evolves
    
    #Get trial by trial likelihoods for both sources - p(x_t|z)
    z_lik = np.array([x,x])
    z_lik[0,x == 0] = .8
    z_lik[0,x == 1] = .2
    z_lik[1,x == 0] = .2
    z_lik[1,x == 1] = .8
    
    #Loop trhough trials and get trial by trial model source estimates
    z_hat = np.empty(len(x))  #Model trial by trial estimate of source
    for i in range(len(x)):
        # On the first trial set to uniform probability over both sources
        if i == 0:
            pz[:,i] = np.array([.5,.5])
            
        #Else, update estimates with each new piece of evidence (x)
        else:
            for iz in range(2):
                pz[iz,i] = z_lik[iz,i-1]*pz[iz,i-1]
        
        #Normalize
        pz[:,i] = pz[:,i]/np.sum(pz[:,i])
        z_hat[i] = sum(zspc*pz[:,i])
    
    return(pz,z_hat)

In [81]:
def genBinStim(ntrials,p=.8,H=0,K=0):
    """
    Function to generate stimuli of increasing complexity
    Arguments:
        - ntrials: number of observations (trials) to be simulated
        
    Keyword arguments:
        - p: bernoulli probability that source will generate an observation
        - H: hazard rate
            - if scalar, single hazard rate used throughout the task
            - if array, samples from array with replacement with probability k
        - K: meta hazard rate
    
    Returns:
        - obs: pandas data frame with current trial, observation, source, and hazard rate
    """
    #Generate observations
    x = np.empty(ntrials)
    z = np.empty(ntrials)
    h = np.empty(ntrials)
    
    #Space of possible sources (z)
    zspc = np.array([0,1])
    
    #Loop through trials
    for i in range(ntrials):
        
        #Set state and hazard variables
        if i == 0:
            z[i] = np.random.choice(zspc)
            
            #Select one hazard rate at random from input hazard space
            if isinstance(H,int) == False & isinstance(H,float) == False:
                h[i] = np.random.choice(H)
            else:
                h[i] = H
        else:
            #Update hazard space based on meta hazard rate 
            if np.random.uniform(1)<K:
                if isinstance(H,int) == False & isinstance(H,float) == False: 
                    h[i] = np.random.choice(H[H != h[i-1]])
            else:
                h[i] = h[i-1]
                
            
            #Update state space based on current hazard rate
            if np.random.uniform(1) < h[i]:
                z[i] = zspc[zspc!=z[i-1]][0]
            else:
                z[i] = z[i-1]
        
        #Gererate observation
        x[i] = z[i]
        if np.random.uniform(1)>p:
            x[i] = zspc[zspc!=z[i]][0]
    
    #Put everything into a data frame
    obs = pd.DataFrame()
    obs['Trial'] = range(1,ntrials+1)
    obs['H'] = h
    obs['Z'] = z
    obs['X'] = x
    return(obs)

print(genBinStim(100))

    Trial    H    Z    X
0       1  0.0  0.0  1.0
1       2  0.0  0.0  1.0
2       3  0.0  0.0  1.0
3       4  0.0  0.0  1.0
4       5  0.0  0.0  1.0
5       6  0.0  0.0  1.0
6       7  0.0  0.0  1.0
7       8  0.0  0.0  1.0
8       9  0.0  0.0  1.0
9      10  0.0  0.0  1.0
10     11  0.0  0.0  1.0
11     12  0.0  0.0  1.0
12     13  0.0  0.0  1.0
13     14  0.0  0.0  1.0
14     15  0.0  0.0  1.0
15     16  0.0  0.0  1.0
16     17  0.0  0.0  1.0
17     18  0.0  0.0  1.0
18     19  0.0  0.0  1.0
19     20  0.0  0.0  1.0
20     21  0.0  0.0  1.0
21     22  0.0  0.0  1.0
22     23  0.0  0.0  1.0
23     24  0.0  0.0  1.0
24     25  0.0  0.0  1.0
25     26  0.0  0.0  1.0
26     27  0.0  0.0  1.0
27     28  0.0  0.0  1.0
28     29  0.0  0.0  1.0
29     30  0.0  0.0  1.0
..    ...  ...  ...  ...
70     71  0.0  0.0  1.0
71     72  0.0  0.0  1.0
72     73  0.0  0.0  1.0
73     74  0.0  0.0  1.0
74     75  0.0  0.0  1.0
75     76  0.0  0.0  1.0
76     77  0.0  0.0  1.0
77     78  0.0  0.0  1.0


In [ ]:
ntrials = 100     #number of trials
p0 = .2           #probability that source one will generate stim
x0 = np.zeros(ntrials)                      #Observation vector 
x0[np.random.uniform(size=ntrials)<p0] = 1  #Randomly generate observations

pz,z_hat = source_inf(x,p)

figure(1,figsize=(10,4))
plot(range(1,len(x)+1),x,'ob',label='Observation (x)')
plot(range(1,len(x)+1),z_hat,'-b',label='Model Source Estimate (z)')
xlabel('Trials')
title('Source 0 generating stim')
legend()

In [80]:
zspc = np.array([0,1])

zspc[zspc!=1][0]


0